In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
import time
import json

warnings.filterwarnings("ignore")


plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline

holidays = pd.Series(json.load(open(r'../../Data/Holidays.json'))['holidays'])
subwayinfo = pd.read_csv(r'../../Data/BeijingSubway.csv')

In [2]:
df_copy = pd.read_csv(
    r'../../Data/data_set_phase1/train_queries.csv')
df = df_copy.copy()

# Datetime

In [3]:
df['req_time'] = pd.to_datetime(df['req_time'])
df['day_of_week'] = df['req_time'].dt.day_name()
df['req_date'] = df['req_time'].dt.strftime('%m-%d')
df['req_hour'] = df['req_time'].dt.hour
df['req_minute'] = df['req_time'].dt.minute
df['if_holiday'] = (df['req_date'].isin(holidays)).astype(int)

# Week day
current_c = list(df.columns.values)
weekdays = ['Monday', 'Tuesday', 'Wednesday',
            'Thursday', 'Friday', 'Saturday', 'Sunday']
current_c.extend(weekdays)
df = df.reindex(columns=current_c, fill_value=0)
for day_name in weekdays:
    df.loc[(df['day_of_week'] == day_name), [day_name]] = 1

# Hour
current_c = list(df.columns.values)
oclock = [str(i)+'_oclock' for i in range(0, 24)]
current_c.extend(oclock)
df = df.reindex(columns=current_c, fill_value=0)
for h in oclock:
    df.loc[(df['req_hour'] == int(h.split('_')[0])), [h]] = 1

In [8]:
df['date_d'] = df['req_time'].dt.strftime('%d').astype(int)


current_c = list(df.columns.values)
date = [str(i)+'_date' for i in range(1, 32)]
current_c.extend(date)
df = df.reindex(columns=current_c, fill_value=0)
for d in date:
    df.loc[(df['date_d'] == int(d.split('_')[0])), [d]] = 1
print('date Done.')

date Done.


In [9]:
df.head()

,sid,pid,req_time,o,d,day_of_week,req_date,req_hour,req_minute,if_holiday,...,22_date,23_date,24_date,25_date,26_date,27_date,28_date,29_date,30_date,31_date
0,3000821,NaN,2018-11-02 17:54:30,"116.29,39.97","116.32,39.96",Friday,11-02,17,54,0,...,0,0,0,0,0,0,0,0,0,0
1,3085857,210736.0,2018-11-16 10:53:10,"116.39,39.84","116.33,39.79",Friday,11-16,10,53,0,...,0,0,0,0,0,0,0,0,0,0
2,2944522,NaN,2018-10-06 10:33:58,"116.31,39.93","116.27,40.00",Saturday,10-06,10,33,1,...,0,0,0,0,0,0,0,0,0,0
3,559931,202427.0,2018-11-23 14:54:11,"116.27,39.88","116.39,39.90",Friday,11-23,14,54,0,...,0,1,0,0,0,0,0,0,0,0
4,2819352,172251.0,2018-10-30 11:48:41,"116.34,39.96","116.37,39.86",Tuesday,10-30,11,48,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
df.drop(columns=['day_of_week','req_date', 'req_hour'], inplace=True)

# Distance

In [ ]:
df['o_lng'] = df['o'].apply(lambda x: float(x.split(',')[0]))
df['o_lat'] = df['o'].apply(lambda x: float(x.split(',')[1]))
df['d_lng'] = df['d'].apply(lambda x: float(x.split(',')[0]))
df['d_lat'] = df['d'].apply(lambda x: float(x.split(',')[1]))
df['od_manhattan_distance'] = abs(
    df['o_lng']-df['d_lng'])+abs(df['o_lat']-df['d_lat'])

In [ ]:
df['o_nearest_dis'] = np.nan
df['d_nearest_dis'] = np.nan

In [ ]:
for index in df.index:
    df[index, 'o_nearest_dis'] = (abs(subwayinfo['station_longitude']-df.loc[index]
                                      ['o_lng'])+abs(subwayinfo['station_latitude']-df.loc[index]['o_lat'])).min()
    df[index, 'd_nearest_dis'] = (abs(subwayinfo['station_longitude']-df.loc[index]
                                      ['d_lng'])+abs(subwayinfo['station_latitude']-df.loc[index]['d_lat'])).min()

In [ ]:
df.head()